In [2]:
import sys
import os
import numpy as np
from pathlib import Path
os.chdir(Path(os.getcwd()).resolve().parents[1])
import setup
from methods import grid_selection_amanda_dynamic
from sklearn.model_selection import ParameterGrid
import time
from multiprocessing import Process, Value, Lock


class Counter(object):
    def __init__(self, initval=0):
        self.val = Value('i', initval)
        self.lock = Lock()

    def increment(self):
        with self.lock:
            self.val.value += 1

    def value(self):
        with self.lock:
            return self.val.value

def func(counter):
    for i in range(50):
        time.sleep(0.01)
        counter.increment()

def writeResults(datasetID, bestScore, bestParams, clfName, distanceMetric, beta=None):
    if distanceMetric == 'BBD' and isinstance(beta,list):
        path = "results/batch/dynamic/gridsearch_amanda_dynamic_entireData_{}_EVL-{}-ALLBETAS.txt".format(distanceMetric,clfName)
    elif distanceMetric == 'BBD':
        path = "results/batch/dynamic/gridsearch_amanda_dynamic_entireData_{}{:.3f}_EVL-{}.txt".format(distanceMetric, beta,clfName)
    else:
        path = "results/batch/dynamic/gridsearch_amanda_dynamic_entireData_{}_EVL-{}.txt".format(distanceMetric,clfName)
        
    file = open(path,"a") 
    string = "{}: {} using {} \n".format(datasetID, bestScore, bestParams)
    file.write(string)
    file.close() 


def main(counter, distanceMetric, beta, total):
    is_windows = sys.platform.startswith('win')
    sep = '\\'
    
    if is_windows == False:
        sep = '/'

    path = os.getcwd()+sep+'data'+sep
    
    #loading datasets
    #datasets = [setup.loadCDT, setup.loadCHT, setup.load2CDT, setup.load2CHT, setup.load4CR, setup.load4CRE_V1, 
    #            setup.load4CRE_V2, setup.load5CVT, setup.loadCSurr, setup.load4CE1CF, setup.loadUG_2C_2D, setup.loadMG_2C_2D, 
    #            setup.loadFG_2C_2D, setup.loadUG_2C_3D, setup.loadUG_2C_5D, setup.loadGEARS_2C_2D, setup.loadCheckerBoard, 
    #            setup.loadElecData, setup.loadNOAADataset, setup.loadKeystroke]
    #datasets = [setup.loadUG_2C_3D, setup.loadUG_2C_5D, setup.loadGEARS_2C_2D, setup.loadCheckerBoard, 
    #            setup.loadElecData, setup.loadNOAADataset, setup.loadKeystroke]
    batches=100
    #datasets=[setup.loadKeystroke]
    #batches=8
    #datasets=[setup.loadNOAADataset]
    #batches=50
    
    arrClfName = ['LP']#['SGD', 'NB', 'RF', 'LP', 'KNN']
    #distanceMetric = 'BBD'
    #beta = [-50,-10, -5, -2, -1, -0.5, -0.1, -0.01, -0.001, 1.001, 1.01, 1.1, 1.5, 2, 3, 5, 10, 50]
    allBetas=False
    for clfName in arrClfName:
        if distanceMetric == 'BBD' and allBetas:
            print("**************** BEGIN of {}-{} results ****************".format(clfName, distanceMetric))
        elif distanceMetric == 'BBD':
            print("**************** BEGIN of {}-{}{:.3f} results ****************".format(clfName, distanceMetric, beta))            
        else:
            print("**************** BEGIN of {}-{} results ****************".format(clfName, distanceMetric))
        sys.stdout.flush()

        batches=47
        poolSize = None
        isBatchMode = True
        #testing grid search
        for i in range(len(datasets)):
            batches=47
            if i==len(datasets)-2:
                batches=24
            elif i==len(datasets)-1:
                batches=4
            
            finalScore = 0
            best_grid={}
            dataValues, dataLabels, description = datasets[i](path, sep)

            #Train-test split
            availableQty = int(0.5*len(dataLabels))
            availableLabels = dataLabels[:availableQty] 
            availableData = dataValues[:availableQty]

            # 70/30 train/test data
            initialLabeledData = int(0.1*len(availableLabels))
            sizeOfBatch = int((len(availableLabels)-initialLabeledData)/batches)

            #print("{}: {} batches of {} instances".format(description, batches, sizeOfBatch))

            tuned_params = [{"sizeOfBatch":[sizeOfBatch], 
                             "batches":[batches], "poolSize":[poolSize], "isBatchMode":[isBatchMode], 
                             "initialLabeledData":[initialLabeledData], "clfName":[clfName],
                            "distanceMetric":[distanceMetric], "beta":[beta]}]
            if clfName == 'LP' or clfName == 'KNN':
                tuned_params[0].update({"K":[2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]})

            for g in ParameterGrid(tuned_params):
                averageAccuracy=0
                gs = grid_selection_amanda_dynamic.run(**g)

                try:
                    gs.fit(availableData, availableLabels)
                    averageAccuracy = np.mean(gs.predict())
                    print(averageAccuracy, g)
                    if finalScore < averageAccuracy:
                        finalScore = averageAccuracy
                        best_grid = g
                except Exception:
                    print("An error occured in ", description, g)
                    #raise Exception

            #print(finalScore)
            #print(best_grid)
            #print("=======================================================================================================")
            counter.increment()
            print(("{:.2f}% completed").format((counter.value()*100)/total))
            sys.stdout.flush()

            writeResults(description, finalScore, best_grid, clfName, distanceMetric, beta)
        
        if distanceMetric == 'BBD' and allBetas:
            print("**************** BEGIN of {}-{} results ****************".format(clfName, distanceMetric))
        elif distanceMetric == 'BBD':
            print("******** END of {}-{}{:.3f} results ********".format(clfName, distanceMetric, beta))
        else:
            print("******** END of {}-{} results ********".format(clfName, distanceMetric))
        sys.stdout.flush()
    
if __name__ == "__main__":
    
    distanceMetrics = ['BBD']
    betas = [-50,-10, -5, -2, -1, -0.5, -0.1, -0.01, -0.001, 1.001, 1.01, 1.1, 1.5, 2, 3, 5, 10, 50]
    datasets = [setup.loadCDT, setup.loadCHT, setup.load2CDT, setup.load2CHT, setup.load4CR, setup.load4CRE_V1, 
                setup.load4CRE_V2, setup.load5CVT, setup.loadCSurr, setup.load4CE1CF, setup.loadUG_2C_2D, setup.loadMG_2C_2D, 
                setup.loadFG_2C_2D, setup.loadUG_2C_3D, setup.loadUG_2C_5D, setup.loadGEARS_2C_2D, setup.loadCheckerBoard, 
                setup.loadElecData, setup.loadNOAADataset, setup.loadKeystroke]
    total = len(distanceMetrics) * len(betas)
    
    counter = Counter(0)
    procs =[]
    for b in betas:
        for d in distanceMetrics:
            procs.append(Process(target=main, args=(counter, d, b, datasets, total)))

    for p in procs: p.start()
    for p in procs: p.join()
